In [45]:
import pandas as pd
import numpy as np

# Q1
We want to look at data for the Facebook, Apple, Amazon, Netflix, and Google (FAANG) stocks, but we were given each as a separate CSV file (obtained using the stock_analysis package we will build in Chapter 7, Financial Analysis – Bitcoin and the Stock Market). Combine them into a single file and store the dataframe of the FAANG data as faang for the rest of the exercises:

1) Read in the aapl.csv, amzn.csv, fb.csv, goog.csv, and nflx.csv files.
2) Add a column to each dataframe, called ticker, indicating the ticker symbol it is for (Apple's is AAPL, for example); this is how you look up a stock. In this case, the filenames happen to be the ticker symbols.
3) Append them together into a single dataframe.
4) Save the result in a CSV file called faang.csv.

In [23]:
faang = pd.DataFrame()
for ticker in ['fb', 'aapl', 'amzn', 'nflx', 'goog']:
    df = pd.read_csv(f'exercises/{ticker}.csv')
    # make the ticker the first column
    df.insert(0, 'ticker', ticker.upper())
    faang = pd.concat([faang, df])

faang.to_csv('faang.csv', index=False)
faang.head(3)

,ticker,date,high,low,open,close,volume
0,FB,2018-01-02,181.580002,177.550003,177.679993,181.419998,18151900.0
1,FB,2018-01-03,184.779999,181.330002,181.880005,184.669998,16886600.0
2,FB,2018-01-04,186.210007,184.100006,184.899994,184.330002,13880900.0


In [24]:
faang.dtypes

ticker     object
date       object
high      float64
low       float64
open      float64
close     float64
volume    float64
dtype: object

With faang, use type conversion to cast the values of the date column into datetimes and the volume column into integers. Then, sort by date and ticker.

In [25]:
faang = faang.assign(
    date = pd.to_datetime(faang.date),
    volume = faang.volume.astype("int")
)
faang.dtypes

ticker            object
date      datetime64[ns]
high             float64
low              float64
open             float64
close            float64
volume             int64
dtype: object

Find the seven rows in faang with the lowest value for volume.

In [26]:
faang.nsmallest(n=7, columns="volume")

,ticker,date,high,low,open,close,volume
126,GOOG,2018-07-03,1135.819946,1100.020020,1135.819946,1102.890015,679000
226,GOOG,2018-11-23,1037.589966,1022.398987,1030.000000,1023.880005,691500
99,GOOG,2018-05-24,1080.469971,1066.150024,1079.000000,1079.239990,766800
130,GOOG,2018-07-10,1159.589966,1149.589966,1156.979980,1152.839966,798400
152,GOOG,2018-08-09,1255.541992,1246.010010,1249.900024,1249.099976,848600
159,GOOG,2018-08-20,1211.000000,1194.625977,1205.020020,1207.770020,870800
161,GOOG,2018-08-22,1211.839966,1199.000000,1200.000000,1207.329956,887400


Right now, the data is somewhere between long and wide format. Use melt() to make it completely long format. Hint: date and ticker are our ID variables (they uniquely identify each row). We need to melt the rest so that we don't have separate columns for open, high, low, close, and volume.

In [28]:
faang.melt(
    id_vars=["ticker", "date"],
    value_vars=["high", "low", "open", "close", "volume"],
    var_name="OHLC",
    value_name="Value"
)

,ticker,date,OHLC,Value
0,FB,2018-01-02,high,1.815800e+02
1,FB,2018-01-03,high,1.847800e+02
2,FB,2018-01-04,high,1.862100e+02
3,FB,2018-01-05,high,1.869000e+02
4,FB,2018-01-08,high,1.889000e+02
...,...,...,...,...
6270,GOOG,2018-12-24,volume,1.590300e+06
6271,GOOG,2018-12-26,volume,2.373300e+06
6272,GOOG,2018-12-27,volume,2.109800e+06
6273,GOOG,2018-12-28,volume,1.414800e+06


Suppose we found out that on July 26, 2018 there was a glitch in how the data was recorded. How should we handle this? Note that there is no coding required for this exercise.

In [29]:
faang[faang.date == "2018-07-26"]

,ticker,date,high,low,open,close,volume
142,FB,2018-07-26,180.130005,173.750000,174.889999,176.259995,169803700
142,AAPL,2018-07-26,48.990002,48.402500,48.652500,48.552502,76304000
142,AMZN,2018-07-26,1844.680054,1804.500000,1839.000000,1808.000000,9924400
142,NFLX,2018-07-26,365.540009,356.630005,358.190002,363.089996,6993700
142,GOOG,2018-07-26,1269.770996,1249.020020,1251.000000,1268.329956,2405600


The European Centre for Disease Prevention and Control (ECDC) provides an open dataset on COVID-19 cases called daily number of new reported cases of COVID-19 by country worldwide (https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide). This dataset is updated daily, but we will use a snapshot that contains data from January 1, 2020 through September 18, 2020. Clean and pivot the data so that it is in wide format:
1) Read in the covid19_cases.csv file.
2) Create a date column using the data in the dateRep column and the pd.to_datetime() function.
3) Set the date column as the index and sort the index.
4) Replace all occurrences of United_States_of_America and United_Kingdom with USA and UK, respectively. Hint: the replace() method can be run on the dataframe as a whole.
5) Using the countriesAndTerritories column, filter the cleaned COVID-19 cases data down to Argentina, Brazil, China, Colombia, India, Italy, Mexico, Peru, Russia, Spain, Turkey, the UK, and the USA.
6) Pivot the data so that the index contains the dates, the columns contain the country names, and the values are the case counts (the cases column). Be sure to fill in NaN values with 0.

In [30]:
covid = pd.read_csv("exercises/covid19_cases.csv")
covid.head(3)

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
0,01/01/2020,1,1,2020,0,0,Lithuania,LT,LTU,2794184.0,Europe,NaN
1,01/01/2020,1,1,2020,0,0,Iceland,IS,ISL,356991.0,Europe,NaN
2,01/01/2020,1,1,2020,0,0,Nepal,NP,NPL,28608715.0,Asia,NaN


In [35]:
covid["date"] = pd.to_datetime(covid["dateRep"], format="%d/%m/%Y")
covid = covid.set_index("date")
covid.head(3)

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
date,,,,,,,,,,,,
2020-01-01,01/01/2020,1,1,2020,0,0,Lithuania,LT,LTU,2794184.0,Europe,NaN
2020-01-01,01/01/2020,1,1,2020,0,0,Iceland,IS,ISL,356991.0,Europe,NaN
2020-01-01,01/01/2020,1,1,2020,0,0,Nepal,NP,NPL,28608715.0,Asia,NaN


In [ ]:
covid = covid.replace("United_States_of_America", "USA")
covid = covid.replace("United_Kingdom", "UK")

In [40]:
filter_countries = [
    "Argentina", 
    "Brazil", 
    "China", 
    "Colombia", 
    "India", 
    "Italy", 
    "Mexico", 
    "Peru", 
    "Russia", 
    "Spain", 
    "Turkey", 
    "UK", 
    "USA"
]
covid = covid[covid["countriesAndTerritories"].isin(filter_countries)]
covid.head(3)

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
date,,,,,,,,,,,,
2020-01-01,01/01/2020,1,1,2020,0,0,India,IN,IND,1.366418e+09,Asia,NaN
2020-01-01,01/01/2020,1,1,2020,0,0,China,CN,CHN,1.433784e+09,Asia,NaN
2020-01-01,01/01/2020,1,1,2020,0,0,Mexico,MX,MEX,1.275755e+08,America,NaN


In [ ]:
covid.pivot(columns=["countriesAndTerritories"], values="cases").replace(np.nan, 0) # or .fillna(0)

countriesAndTerritories,Argentina,Brazil,China,Colombia,India,Italy,Mexico,Peru,Russia,Spain,Turkey,UK,USA
date,,,,,,,,,,,,,
2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-03,0.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-05,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-14,10778.0,14768.0,29.0,7355.0,92071.0,1456.0,4408.0,6787.0,5449.0,27404.0,1527.0,3330.0,33871.0
2020-09-15,9056.0,15155.0,22.0,5573.0,83809.0,1008.0,3335.0,4241.0,5509.0,9437.0,1716.0,2621.0,34841.0
2020-09-16,9908.0,36653.0,24.0,6698.0,90123.0,1229.0,4771.0,4160.0,5529.0,11193.0,1742.0,3103.0,51473.0


In order to determine the case totals per country efficiently, we need the aggregation skills we will learn about in Chapter 4, Aggregating Pandas DataFrames, so the ECDC data in the covid19_cases.csv file has been aggregated for us and saved in the covid19_total_cases.csv file. It contains the total number of cases per country. Use this data to find the 20 countries with the largest COVID-19 case totals. Hints: when reading in the CSV file, pass in index_col='cases', and note that it will be helpful to transpose the data before isolating the countries.

In [56]:
covid_total = pd.read_csv("exercises/covid19_total_cases.csv", index_col="index").T
covid_total

index,cases
Afghanistan,38919
Albania,12073
Algeria,49413
Andorra,1564
Angola,3789
...,...
Vietnam,1068
Western_Sahara,766
Yemen,2024
Zambia,14022


In [59]:
covid_total.nlargest(20, columns="cases")

index,cases
USA,6724667
India,5308014
Brazil,4495183
Russia,1091186
Peru,756412
Colombia,750471
Mexico,688954
South_Africa,657627
Spain,640040
Argentina,601700
